## Get jurisdiction geography for parcels (Nat query, April 2016)

In [3]:
import pandas as pd

### Load parcels table from HDFStore

In [105]:
path = '/Users/smmaurer/Desktop/MTC BAUS data/2015_09_01_bayarea_v3.h5'
with pd.HDFStore(path) as hdf:
    print hdf.keys()

['/buildings', '/households', '/jobs', '/parcels', '/zones']


In [106]:
parcels = pd.read_hdf(path, 'parcels')

In [107]:
parcels.index.name

'parcel_id'

In [108]:
parcels.count()

development_type_id         1956212
land_value                  1956212
acres                       1956212
county_id                   1956212
zone_id                     1956212
proportion_undevelopable    1956212
tax_exempt_status           1956212
apn                         1956212
parcel_id_local             1956212
geom_id                     1956212
imputation_flag             1956212
x                           1956208
y                           1956208
shape_area                  1956212
dtype: int64

In [46]:
parcels.geom_id.nunique()

1956212

### Load mapping from geom id to jurisdiction id

In [47]:
path = '/Users/smmaurer/Desktop/MTC BAUS data/02_01_2016_parcels_geography.csv'
geodf = pd.read_csv(path, index_col="geom_id", dtype={'jurisdiction': 'str'})

In [48]:
geodf.index.name

'geom_id'

In [49]:
geodf.count()

jurisdiction_id    1956208
pda_id              264205
tpp_id              891455
exp_id               55306
opp_id                 442
zoningmodcat       1956208
perffoot           1956208
perfarea           1956208
dtype: int64

### Load mapping from jurisdiction id to name

In [116]:
path = '/Users/smmaurer/Dropbox/Git-rMBP/ual/bayarea_urbansim/data/census_id_to_name.csv'
namedf = pd.read_csv(path)

In [117]:
namedf['jurisdiction_id'] = namedf.census_id

In [118]:
namedf.index.name

In [122]:
namedf.count()

census_id          109
name10             109
jurisdiction_id    109
dtype: int64

### Join everything together

In [109]:
parcels['geom_id'].reset_index().describe()

,parcel_id,geom_id
count,1956212.000000,1.956212e+06
mean,1018504.384993,-6.347338e+11
std,597284.502117,5.076224e+12
min,1.000000,2.054503e+06
25%,499974.750000,4.405522e+12
50%,1005873.500000,8.793826e+12
75%,1544654.250000,1.318756e+13
max,2054506.000000,1.759217e+13


In [131]:
merged = pd.merge(parcels['geom_id'].reset_index(), 
                  geodf['jurisdiction_id'].reset_index(), 
                  how='left', on='geom_id')

In [132]:
merged = pd.merge(merged, namedf[['jurisdiction_id', 'name10']], 
                  how='left', on='jurisdiction_id').set_index('parcel_id')

In [133]:
print merged.head()

                  geom_id  jurisdiction_id         name10
parcel_id                                                
229116     10305106092872            41992      Livermore
244166     11107351665227            41992      Livermore
202378     11030175960628            33000        Hayward
2004420     6381677629073               97  Sonoma County
340332       314875459798            26000        Fremont


In [134]:
merged.count()

geom_id            1956212
jurisdiction_id    1956208
name10             1956208
dtype: int64

In [136]:
merged.geom_id.nunique()

1956212

In [137]:
merged.describe()

,geom_id,jurisdiction_id
count,1.956212e+06,1956208.000000
mean,-6.347338e+11,43952.457649
std,5.076224e+12,27783.646091
min,2.054503e+06,1.000000
25%,4.405522e+12,17610.000000
50%,8.793826e+12,53000.000000
75%,1.318756e+13,68000.000000
max,1.759217e+13,86930.000000


In [135]:
merged.to_csv('parcel_jurisdictions_v1.csv')